In [1]:
def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import plotly.graph_objects as go
    import pickle
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df["user"]==user_id]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/week_day_hour_minutes.pickle', 'rb') as handle:
        day_hour_duration = pickle.load(handle)
    if user_id in day_hour_duration:
        user_day_hour_duration=day_hour_duration[user_id]
    
    if len(user_df) > 0:
        avg_number_of_stressors=0
        average_stress_duration=0
        number_of_stressors_this_week=0
        stress_duration_this_week=0

        user_df_this_week=user_df.loc[user_df["week"] == week_num]
        if len(user_df_this_week)>0:
            users_day_hour_duration_user_week=user_day_hour_duration[week_num]        
            sum_of_stressed_minutes_this_week=0
            for day in users_day_hour_duration_user_week:
                sum_of_stressed_minutes_this_week+=sum(users_day_hour_duration_user_week[day].values())
            number_of_days=len(users_day_hour_duration_user_week)
            stress_duration_this_week=user_df_this_week['duration'].sum()/sum_of_stressed_minutes_this_week * 60
            temp1_stressors=user_df_this_week.groupby(['Stressor'], as_index=False)["counter"].count()  
            number_of_stressors_this_week=temp1_stressors['counter'].sum()
        else:
            stress_duration_this_week=0
            number_of_stressors_this_week=0

#         user_df_prev_week=user_df.loc[user_df["week"] != week_num]
        if len(user_df)!=0:
            sum_of_stressed_minutes_all_weeks=0
            for week in user_day_hour_duration:
#                 if week !=week_num:
                    for day in user_day_hour_duration[week]:
                        sum_of_stressed_minutes_all_weeks+=sum(user_day_hour_duration[week][day].values())
            average_stress_duration=user_df['duration'].sum()/sum_of_stressed_minutes_all_weeks*60
            temp2_stressors=user_df.groupby(['Stressor'], as_index=False)["counter"].count()        
            avg_number_of_stressors=int(temp2_stressors['counter'].sum()/len(user_df.week.unique()))
        else:
            average_stress_duration=stress_duration_this_week
            avg_number_of_stressors=number_of_stressors_this_week           
        
#         user_df_prev_week=user_df.loc[user_df["week"] != week_num]
#         if len(user_df_prev_week)!=0:
#             sum_of_stressed_minutes_prev_weeks=0
#             for week in user_day_hour_duration:
#                 if week !=week_num:
#                     for day in user_day_hour_duration[week]:
#                         sum_of_stressed_minutes_prev_weeks+=sum(user_day_hour_duration[week][day].values())
#             average_stress_duration=user_df_prev_week['duration'].sum()/sum_of_stressed_minutes_prev_weeks*60
#             temp2_stressors=user_df_prev_week.groupby(['Stressor'], as_index=False)["counter"].count()        
#             avg_number_of_stressors=int(temp2_stressors['counter'].sum()/len(user_df_prev_week.week.unique()))
#         else:
#             average_stress_duration=stress_duration_this_week
#             avg_number_of_stressors=number_of_stressors_this_week

        range_of_stressor=0
        if avg_number_of_stressors <1:
            range_of_stressor=4
        else:
            range_of_stressor=avg_number_of_stressors
        color="#008000"
        if stress_duration_this_week <= average_stress_duration:
            color="#008000"
        else:
            color="#ff0000"

        fig23 = go.Figure()

        fig23.add_trace(go.Indicator(
            mode = "number",
            value = round(average_stress_duration,1),
            domain = {'row': 0, 'column': 0},
             title ={"text":"Stressed minutes per hour (Cumulative)","font_size":12}))



        fig23.add_trace(go.Indicator(
            value = round(stress_duration_this_week,1),
            delta = {'reference': round(average_stress_duration,1)},
           gauge = {'bar_color':color,
                    'shape':"angular",
                'axis' : {'visible': True,'range':(0,60)},
            'threshold': {
                    'line': {'color': "black", 'width': 4},
                    'thickness': 0.9,
                    'value': round(average_stress_duration,1)}},
            domain = {'row': 1, 'column': 0}))

        fig23.add_trace(go.Indicator(
            mode = "number",
            value = avg_number_of_stressors,
            domain = {'row': 2, 'column': 0},
             title ={"text":"# Stressors per Week (Cumulative)",'font_size':12}))

        fig23.add_trace(go.Indicator(
            mode = "number+delta+gauge",
            value = number_of_stressors_this_week,
            domain = {'row': 3, 'column': 0},
            gauge = {'bar_color':color,
                    'shape':"angular",
                'axis' : {'visible': True,'range':(0,2*range_of_stressor)},
            'threshold': {
                    'line': {'color': "black", 'width': 4},
                    'thickness': 0.9,
                    'value': avg_number_of_stressors}},
             title ={"text":"# Stressors (Week " + str(week_num) + ")",'font_size':12}))



        fig23.update_layout( 
    #             title={'text':"<b>Overview</b>",
    #                  'y':0.95,
    #                 'x':0.5,
    #                   'xanchor': 'center',
    #                 'yanchor': 'top'},
    #              title_font_color="black",
    #              title_font_family="Times New Roman",
    #             title_font_size=30,
            font_size=10,
            grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
            template = {'data' : {'indicator': [{
                'title': {'text': "Stressed Minutes per hour (Week " + str(week_num) + ")",'font_size':12},
                'mode' : "number+delta+gauge",
                'delta' : {'reference': avg_number_of_stressors}}]
                                 }})


        fig23.update_traces(delta_increasing={"color":"#ff0000"},delta_decreasing={"color":"#008000"},gauge_axis_tickmode="auto")
        fig23.update_layout(
                    autosize=True,
                   
                    margin=dict(
                        l=0,
                        r=0,
                        b=10,
                        t=20,

                    ),)
        data_summary_user=data_summary_user_current["Overall Average"]

        return "Overall Average",fig23,data_summary_user

    else:
        return "","",""
